In [45]:
import os
import pandas as pd
import re
import numpy as np


def get_filename(path):
    filename = os.path.basename(path)
    name = re.split(r'[_\.]', filename)[0]

    key = name[:3]
    return key

def get_peptide_isr(folder_path):
    c_peptide_values = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.lower().endswith("txt") and not f.lower().endswith(("_re.txt", "_r.txt",))]
    isr_values = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.lower().endswith(("_re.txt", "_r.txt"))]


    c_peptide_database = {}
    isr_values_database = {}

    for file in c_peptide_values:

        key = get_filename(file)

        if key not in c_peptide_database:
            c_peptide_database[key] = []


        c_peptide_database[key].append(file)

    for file in isr_values:
        key = file


        if key not in isr_values_database:
            isr_values_database[key] = []


        isr_values_database[key].append(file)

    return {"C_peptide": c_peptide_database, "ISR": isr_values_database}

path_to_folder = "/Users/alert/Desktop/ISEC/ISR-PEPTIDE-TXT/"



def preprocess_file(filename, col_name=None):
    with open(filename, "r") as f:
        lines = f.readlines()
    
    # Extract metadata
    metadata = {}
    for line in lines:
        if ":" in line:
            key, value = line.split(":", 1)
            metadata[key.strip()] = value.strip()
    
    df_meta = pd.DataFrame(
        metadata.items(),
        columns=["TIME(min)", "C_PEPTIDE"]
    )

    table_start = 0
    for i, line in enumerate(lines):
        if "TIME (min)" in line and "C-PEPTIDE" in line:
            table_start = i + 1 
            break


    df = pd.read_csv(
        filename,
        sep=r"\s+",
        skiprows=table_start,
        engine="python",
        names=["TIME(min)", "C_PEPTIDE"],
        on_bad_lines='skip'
    )


    new_dataframe  = pd.concat([df, df_meta])
    datasetframe = new_dataframe.reset_index().drop(columns=["index"])

    # Optonally rename the column to avoid conflicts when merging
    if col_name:
        datasetframe.rename(columns={"C_PEPTIDE": col_name}, inplace=True)

    return datasetframe, metadata["ID"]


c_pep = get_peptide_isr(path_to_folder)["C_peptide"]
c_pep_dataframes = {}
for files in c_pep:
    
    final_df = preprocess_file(c_pep[files][0], col_name=f"{preprocess_file(c_pep[files][0])[1]}")[0]
    

    # Step 2: Loop through the rest and merge
    for idx, file in enumerate(c_pep[files][1:], start=2):
        id = preprocess_file(file)[1]
        
        key = get_filename(file)
        df_new = preprocess_file(file, col_name=f"{id}.")[0]
        final_df = pd.merge(final_df, df_new, on="TIME(min)", how="outer")

    key = get_filename(files)

    if key not in c_pep_dataframes:
        c_pep_dataframes[key] = final_df



c_pep_dataframes.keys()






dict_keys(['CON', 'FEB', 'SEP', 'OCT', 'MAR', 'JAN', 'DEC', 'AUG', 'NAP', 'TES', 'M2', 'APR', 'M3', 'M1', 'M4', 'JUL', 'M11', 'DIA', 'M33', 'M51', 'M52'])

In [ ]:
isr = get_peptide_isr(path_to_folder)["ISR"]



# ================================
# STEP 2: PREPROCESS A SINGLE FILE
# ================================

def preprocess_file(filepath):

    with open(filepath, "r") as f:
        lines = f.readlines()

    # Find table start
    table_start = 0
    for i, line in enumerate(lines):
        if "from" in line and "to" in line:
            table_start = i + 1
            break

    # Read table
    df = pd.read_csv(
        filepath,
        sep=r"\s+",
        skiprows=table_start,
        engine="python",
        names=["from", "to", "isr"],
        on_bad_lines="skip"
    )

    # Fix first row
    df.loc[df.index[0], "isr"] = df.loc[df.index[0], "to"]
    df.loc[df.index[0], "to"]  = df.loc[df.index[0], "from"]

    # Stop at first NaN row
    mask = df.isna().any(axis=1)
    first_none_idx = mask.idxmax() if mask.any() else len(df)
    df = df.iloc[:first_none_idx]

    subject=""

    def get_filename(path):
        filename = os.path.basename(path)
        name = re.split(r'[_\]', filename)[0]

        key = name
        return key
    # print(get_filename(isr["CON"]))


for i in isr:
    print(preprocess_file(i))


    # if filename.endswith("RE.TXT"):
    #     subject = filename.replace("_RE.TXT", "")
    # elif filename.endswith("R.TXT"):
    #     subject = filename.replace("_R.TXT", "")
    # else:
    #     subject = subject

    # # Final clean dataframe
    # df = (
    #     df.drop(columns=["from"])
    #       .rename(columns={
    #           "to": "TIME(min)",
    #           "isr": subject
    #       })
    # )

    # return df


# ================================
# STEP 3: MERGE & WRITE TO EXCEL
# ================================

# output_file = "ALL_ISR_GROUPS.xlsx"

# with pd.ExcelWriter(output_file, engine="openpyxl") as writer:

#     for group, group_files in file_database.items():
#         print(f"\nProcessing group: {group}")

#         final_df = None

#         for file in group_files:
#             df = preprocess_file(folder_path, file)

#             if final_df is None:
#                 final_df = df
#             else:
#                 final_df = pd.merge(
#                     final_df,
#                     df,
#                     on="TIME(min)",
#                     how="outer"   # IMPORTANT
#                 )

#         final_df = final_df.sort_values("TIME(min)").reset_index(drop=True)
#         final_df.to_excel(writer, sheet_name=group, index=False)

# print("\n✅ All groups successfully written to Excel.")


TypeError: expected str, bytes or os.PathLike object, not dict